In [1]:
import re
import os
import sys
import pickle
sys.path.append("classes")
from pin import Pin
from delayobject import DelayObject
from node import Node
import numpy as np

with open('../newPickle/node_dictionary.pkl', 'rb') as file:
    node_dictionary = pickle.load(file)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
graph = {}

In [3]:
for node in node_dictionary:
    node_object = node_dictionary[node][0] #object node with above properties
    inputslist=node_object.inputs #inputs in going into the node
    graph[node_object]=[]
    for inputs in inputslist: 
        if inputs in node_dictionary:
            o_object = node_dictionary[inputs][0] #input of node_object object
            if o_object not in graph:
                graph[o_object]=[]
            graph[o_object].append(node_object) #nodedict[key] maps to a list thats y [0]
for node in graph:
    node.fanout = len(graph[node])

In [4]:
wire_graph = {}

In [5]:
for node in node_dictionary:
    node_object=node_dictionary[node][0]
    wire_graph[node_object]=[]
    inputs_list=node_object.inputs
    for inputs in inputs_list:
        if inputs == "1'b0" or inputs == "1'b1" or inputs[0]=="{":
            continue
        input_object=node_dictionary[inputs][0]
        if input_object not in wire_graph:
            wire_graph[input_object]=[]
        wire_graph[input_object].append(node_object)
        
for node in wire_graph:
    if node.cell != "":
        node.load = sum([wire.maxcap for wire in wire_graph[node]])

In [6]:
additional_points = np.linspace(0.9, 1.5, 1000).tolist()
def get_power(value, peak, t1, t2, t):
    c = value + t1
    r2 = c + t2
    r1 = value
    m1 = peak / (c - r1)
    m2 = peak / (c - r2)
    
    if t <= c:
        return m1 * (t - r1)
    return m2 * (t - r2)

def get_power_dict(node,additional_points):
    
    time_powerdict={}
    newdict = node.toggle
    newdict_keys = list(newdict.keys())
    power_list=[] 
    for time in newdict_keys:
        power_list.append(time) #starting time
        power_list.append(time+node.t1+node.t2) #ending time
    power_list = list(set(power_list + additional_points)) # Convert additional_points to a list before adding
    power_list.sort()
    
    for value in newdict_keys:
        if newdict[value] == 1:
            for time in power_list:
                power = get_power(value, node.peak, node.t1, node.t2, time)
                if power >= 0:
                    if time not in time_powerdict:
                        time_powerdict[time] = []
                    time_powerdict[time].append(power)
    for toggle_time in list(time_powerdict.keys()):
        time_powerdict[toggle_time] = sum(time_powerdict[toggle_time])
    return time_powerdict

In [ ]:
# creating time to power value dictionary for each node
c = 0
for node in wire_graph:
    if not node.powertimedict:
        node.powertimedict = get_power_dict(node,additional_points)
        c+=1
    print(c)

0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276


In [8]:
for x in wire_graph:
    x.t1=0.27696088077503833
    x.t2=0.45389178609342473

In [ ]:
def get_input_objects(node):
    li=[]
    for input_node in node.inputs:
        if input_node[0] == "{" or input_node == "1'b1" or input_node == "1'b0":
            continue
        li.append(node_dictionary[input_node][0])
    return li

node_inputs_dictionary = {}

for wires in wire_graph:
    node_inputs_dictionary[wires] = get_input_objects(x)
    
def is_DFF(node):
    return node.cell.find("DFF")>=0

def get_state(node):
    return node_inputs_dictionary[node]

def generate_subset(start_node):
    def dfs(node):
        visited.add(node)
        if is_DFF(node):
            return
        for neighbor in get_state(node):
            if neighbor not in visited:
                dfs(neighbor)

    visited = set()
    dfs(start_node)

    return visited
datas = []
for x in node_inputs_dictionary:
    datas.append(generate_subset(x))

In [ ]:
def graphify(nodes_list):
    graph = {}
    for node in nodes_list:
        if is_DFF(node):
            if node not in graph:
                graph[node] = []
            continue
        graph[node] = node_inputs_dictionary[node]
    return graph

graph_dataset = []
for x in datas:
    graph_dataset.append(graphify(x))

len(graph_dataset)

In [49]:
l = 0
for x in wire_graph:
    if x.powertimedict:
        l+=1
print(l)

1035


In [ ]:
# Assuming additional_points is defined in your code
additional_points = np.linspace(0.9, 1.5, 1000).tolist()

# Pickle the additional_points variable
with open('additional_points.pkl', 'wb') as file:
    pickle.dump(additional_points, file)